# **Aula 11 - Ensembles**

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## **TOC:**
Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) [Métodos de Ensemble](#intro)
- 2) [Random Forest](#random_forest)
- 3) [Adaboost](#ada)
- 4) [Árvores de regressão](#regression)
- 5) [Exercicio](#exercicio)


## 1) **Métodos de Ensemble** <a class="anchor" id="intro"></a>

Há uma classe de algoritmos de Machine Learning, os chamados **métodos de ensemble** que tem como objetivo **combinar as predições de diversos estimadores mais simples** para gerar uma **predição final mais robusta**
    
Os métodos de ensemble são ainda divididos em duas classes:

- **Métodos de bagging**: têm como procedimento geral construir diversos <font color="orange"><b>estimadores independentes</b></font>, e tomar a média de suas predições como a predição final. O principal objetivo do método é reduzir variância, de modo que o modelo final seja melhor que todos os modelos individuais. Ex.: **random forest.**

<br>

- **Métodos de boosting**: têm como procedimento geral a construção de estimadores de forma sequencial, de modo que estimadores posteriores tentam reduzir o viés do estimador conjunto, que leva em consideração estimadores anteriores. Ex.: **adaboost**.

Para mais detalhes, [clique aqui!](https://scikit-learn.org/stable/modules/ensemble.html)

## 2) **Random Forest** <a class="anchor" id="random_forest"></a>

Uma técnica muito interessante baseada em árvores é o **Random Forest**.

Neste modelo, são criadas varias árvores diferentes aleatoriamente, e a predição final é tomada através do voto majoritário de todas as árvores!

<center><img src="https://i.ytimg.com/vi/goPiwckWE9M/maxresdefault.jpg" width=700></center>

O modelo de Random Forest utiliza os conceitos de **bootstraping** e **aggregation** (ou então, o procedimento composto **bagging**) para criar um modelo composto que é melhor que uma única árvore!

<center><img src="https://c.mql5.com/2/33/image1__1.png" width=600></center>

Para maiores detalhes sobre como o modelo funciona, sugiro os vídeos do canal [StatQuest](https://www.youtube.com/watch?v=J4Wdy0Wc_xQ). 

Obs.: toda a [playlist de machine learning](https://www.youtube.com/playlist?list=PLblh5JKOoLUICTaGLRoHQDuF_7q2GfuJF) é muitíssimo interessante, com vídeos super claros e ilustrativos! Além disso, há outros vídeos de estatística que são muito bons! Este é um dos melhores canais no youtube para se aprender de forma clara e descontraída sobre estatística e machine learning!

Aqui, vamos ver o Random Forest em ação!

In [2]:
from my_pipe import create_pipe
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

def make_reports(pipe, X_train, X_test, y_train, y_test):
    y_test_hat = pipe.predict(X_test)
    y_train_hat = pipe.predict(X_train)

    print("TREINO:")
    print(roc_auc_score(y_train, y_train_hat))
    print("TESTE:")
    print(roc_auc_score(y_test, y_test_hat))

df = pd.read_csv('data/bank_full.csv', sep=";")
X = df.drop(columns='y')
y = df['y'].astype("category").cat.codes

# redefinir pra poder alterar novamente
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33, 
                                                    random_state=42)

numerical_columns = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_nominal_columns = ["job", "marital", "poutcome"]
model = RandomForestClassifier(verbose=1, n_jobs=os.cpu_count() - 2)
pipe = create_pipe(model, numerical_columns, categorical_nominal_columns)
pipe.fit(X_train, y_train)
make_reports(pipe, X_train, X_test, y_train, y_test)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.3s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    3.3s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s


TREINO:
1.0
TESTE:
0.6658838753173882


[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.1s finished


In [3]:
from sklearn.model_selection import RandomizedSearchCV
model = RandomForestClassifier(verbose=0, n_jobs= os.cpu_count() - 2)
pipe = create_pipe(model, numerical_columns, categorical_nominal_columns)

param_grid = {'rf__n_estimators' : [100, 200],
              'rf__max_depth' : [4,6,10,15]}

grid = RandomizedSearchCV(pipe, 
                          param_grid,
                          n_iter=5,
                          scoring="roc_auc", 
                          cv=5,  
                          n_jobs=4, 
                          verbose=1)

grid.fit(X_train, y_train)
make_reports(grid, X_train, X_test, y_train, y_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
TREINO:
0.7993742900724886
TESTE:
0.6537033432469022


______

## **Adaboost** <a class="anchor" id="ada"></a>

O Adaboost significa **Adaptive Boosting**, e tem como procedimento geral **a criação sucessiva de árvores de um único nó (stumps - modelos fracos) que utiliza dos erros da árvore anterior para melhorar a próxima árvore**. As predições finais são feitas com base **nos pesos de cada stump**, cuja determinação faz parte do algoritmo.


<center><img src="https://miro.medium.com/max/1744/1*nJ5VrsiS1yaOR77d4h8gyw.png" width=300></center>

De forma resumida, as principais ideias por trás deste algoritmo são:

- O algoritmo cria e combina um conjunto de **modelos fracos** (em geral, stumps);
- Cada stump é criado **levando em consideração os erros do stump anterior**;
- Alguns dos stumps têm **maior peso de decisão** do que outros na predição final;

<br>

<center><img src="https://www.researchgate.net/profile/Zhuo_Wang8/publication/288699540/figure/fig9/AS:668373486686246@1536364065786/Illustration-of-AdaBoost-algorithm-for-creating-a-strong-classifier-based-on-multiple.png" width=500></center>

<center><img src="https://static.packt-cdn.com/products/9781788295758/graphics/image_04_046-1.png" width=400></center>

In [4]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(n_estimators=50)
pipe = create_pipe(model, numerical_columns, categorical_nominal_columns)
pipe.fit(X_train, y_train)
make_reports(pipe, X_train, X_test, y_train, y_test)

TREINO:
0.6588818245560081
TESTE:
0.6582651435655139


In [5]:
from sklearn.model_selection import GridSearchCV

model = AdaBoostClassifier()
pipe = create_pipe(model, numerical_columns, categorical_nominal_columns)

param_grid = {'rf__n_estimators' : [50, 100, 200, 300]}

grid = GridSearchCV(pipe, 
                    param_grid,
                    scoring="roc_auc", 
                    cv=5,  
                    n_jobs=4, 
                    verbose=1)

grid.fit(X_train, y_train)
make_reports(grid, X_train, X_test, y_train, y_test)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
TREINO:
0.664008103564873
TESTE:
0.6618306573946706


## 4) **Árvores de regressão** <a class="anchor" id="regression"></a>

Alguns algoritmos de classificação podem ser utilizados como algoritmos de regressão, inclusive árvores de decisão!

As **árvores de regressão** consistem em funções com valores discretos, similar a uma escada, onde cada degrau é o valor de uma folha. [Aqui](https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html) há detalhes sobre a classe do sklearn; e [aqui](https://www.youtube.com/watch?v=g9c66TUylZ4) está o StatQuest sobre árvores de regressão!

Considere o seguinte dataset:

<center><img src='https://s3-sa-east-1.amazonaws.com/lcpi/800a4332-e709-4ea3-8c24-959c05c8fd65.png' width=500></center>

O algoritmo irá obter os valores do target como sendo **a média dos valores de cada folha da árvore final**. 

Visualmente: 

<center><img src='https://s3-sa-east-1.amazonaws.com/lcpi/64cb4edd-20e1-486a-8fc9-60e60e1485d5.png' width=500></center>

Para a escolha das melhores divisões: 

- o algoritmo percorre a médida entre cada par de pontos das features; 
- define estes valores como divisões (sequencialmente); 
- para cada divisão experimentada, o algoritmo calcula o MSE;
- a melhor divisão é aquela que apresentar o menor erro!

Visualmente:

<center><img src='https://s3-sa-east-1.amazonaws.com/lcpi/be58ac8b-5c59-4b9f-be79-e000d060e9e3.png' width=500></center>

<center><img src='https://s3-sa-east-1.amazonaws.com/lcpi/1f317afd-6119-41a5-849d-cee038403cf2.png' width=500></center>

Outro exemplo de árvore de regressão treinada:

<center><img src="https://res.cloudinary.com/practicaldev/image/fetch/s--YryIJN_o--/c_imagga_scale,f_auto,fl_progressive,h_900,q_auto,w_1600/https://thepracticaldev.s3.amazonaws.com/i/7oxf0e3cggdj9jayxeig.png" width=600></center>

Vamos fazer um modelo de árvore de regressão para precificação de casas!

## 5) **Exercicio** <a class="anchor" id="exercicio"></a>

In [6]:
df = pd.read_csv("data/house_prices.csv")